# 5.1.1 人臉檢測 (Face Detection)

**WBS 5.1.1**: 傳統機器學習人臉檢測方法

本模組深入探討三種經典人臉檢測方法，從理論到實作，涵蓋參數調優、性能分析與實戰應用。

## 學習目標
- 理解 Haar Cascade、LBP Cascade、HOG + SVM 三種檢測器的原理
- 掌握多尺度檢測與參數優化技巧
- 學習人臉檢測的性能優化策略
- 實作單圖、多圖、實時視訊人臉檢測
- 比較不同方法的速度與準確度權衡

## 前置知識
- Python 基礎語法
- NumPy 陣列操作
- OpenCV 圖像讀取與處理
- 滑動窗口與特徵提取概念

## 課程大綱
1. 人臉檢測基礎概念
2. Haar Cascade 人臉檢測
3. LBP Cascade 人臉檢測
4. HOG + SVM 人臉檢測 (dlib)
5. 多尺度檢測與參數優化
6. 性能比較與應用場景
7. 實戰應用
8. 進階主題與深度學習預覽

In [ ]:
# 導入必要的庫
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time
from typing import List, Tuple

# 設置中文顯示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 設置圖像顯示大小
plt.rcParams['figure.figsize'] = (15, 10)

print("OpenCV version:", cv2.__version__)

# 嘗試導入 dlib
try:
    import dlib
    DLIB_AVAILABLE = True
    print("dlib version:", dlib.__version__ if hasattr(dlib, '__version__') else "available")
except ImportError:
    DLIB_AVAILABLE = False
    print("dlib not available (install with: pip install dlib)")

## 1. 人臉檢測基礎概念

### 什麼是人臉檢測？

人臉檢測（Face Detection）是計算機視覺的基礎任務，目標是在圖像中定位人臉的位置。

### 人臉檢測 vs 人臉識別

| 任務 | 目標 | 輸出 | 難度 |
|-----|------|-----|-----|
| 人臉檢測 | 找到人臉位置 | 邊界框 (x, y, w, h) | 中等 |
| 人臉識別 | 識別人臉身份 | 人名/ID | 較高 |
| 人臉對齊 | 標準化人臉姿態 | 關鍵點坐標 | 中等 |
| 人臉驗證 | 判斷是否同一人 | True/False | 較高 |

### 經典檢測方法演進

```
2001: Viola-Jones (Haar Cascade) → 實時檢測突破
2006: LBP Cascade → 更快速度
2005: HOG + SVM → 更高準確度
2015: CNN (Deep Learning) → 準確度飛躍
2017: MTCNN, Dlib CNN → 多任務學習
2020+: RetinaFace, YOLOv5-Face → SOTA性能
```

### 檢測挑戰

1. **姿態變化**: 正臉、側臉、仰視、俯視
2. **光照條件**: 強光、背光、陰影
3. **遮擋問題**: 口罩、眼鏡、頭髮
4. **尺度變化**: 遠處小臉、近處大臉
5. **表情變化**: 微笑、張嘴、閉眼
6. **圖像質量**: 模糊、噪點、低分辨率

## 2. Haar Cascade 人臉檢測

### 原理說明

**Haar Cascade** 由 Viola 和 Jones 於 2001 年提出，是第一個實時人臉檢測算法。

#### 核心概念

1. **Haar-like 特徵**
   - 矩形特徵：黑白區域差值
   - 邊緣特徵：檢測明暗變化
   - 線特徵：檢測線狀結構
   - 四矩形特徵：檢測複雜模式

2. **積分圖 (Integral Image)**
   - 快速計算矩形區域和
   - 時間複雜度：O(1)

3. **AdaBoost 算法**
   - 選擇最佳特徵
   - 組合弱分類器成強分類器

4. **級聯分類器 (Cascade)**
   - 多層過濾器
   - 快速排除非人臉區域
   - 只有通過所有層的才是人臉

#### 優缺點

**優點**:
- 速度快，適合實時應用
- 輕量級，資源消耗少
- OpenCV 內建，易於使用
- 預訓練模型豐富

**缺點**:
- 側臉檢測效果差
- 對光照敏感
- 誤檢率較高
- 無法處理大角度旋轉

In [ ]:
# 載入 Haar Cascade 分類器
# OpenCV 提供多個預訓練模型

# 方法1: 使用 OpenCV 內建路徑
haar_cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'

# 方法2: 使用專案 assets 路徑（如果已複製）
# haar_cascade_path = '../assets/models/haarcascade_frontalface_default.xml'

# 載入分類器
face_cascade = cv2.CascadeClassifier(haar_cascade_path)

# 驗證載入成功
if face_cascade.empty():
    print("Error: Failed to load Haar Cascade classifier")
else:
    print("Haar Cascade classifier loaded successfully")
    print(f"Model path: {haar_cascade_path}")

In [ ]:
# 載入測試圖像
test_image_path = '../assets/images/basic/faces.jpg'

if not Path(test_image_path).exists():
    print(f"Warning: {test_image_path} not found, creating demo image")
    # 創建示範圖像
    img = np.ones((400, 600, 3), dtype=np.uint8) * 200
    cv2.putText(img, "Place test face image at:", (50, 150), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    cv2.putText(img, test_image_path, (50, 200), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1)
else:
    img = cv2.imread(test_image_path)

# 顯示原始圖像
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title('Original Image', fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.show()

print(f"Image shape: {img.shape}")
print(f"Image size: {img.shape[1]} x {img.shape[0]} pixels")

### Haar Cascade 基本檢測

In [ ]:
def detect_faces_haar(image: np.ndarray, 
                       cascade: cv2.CascadeClassifier,
                       scale_factor: float = 1.1,
                       min_neighbors: int = 5,
                       min_size: Tuple[int, int] = (30, 30)) -> Tuple[np.ndarray, List, float]:
    """
    Detect faces using Haar Cascade classifier
    
    Parameters:
    -----------
    image : np.ndarray
        Input image in BGR format
    cascade : cv2.CascadeClassifier
        Loaded Haar Cascade classifier
    scale_factor : float
        Scale factor for multi-scale detection (1.05-1.4)
    min_neighbors : int
        Minimum neighbors for valid detection (3-6)
    min_size : tuple
        Minimum face size (width, height)
        
    Returns:
    --------
    gray : np.ndarray
        Grayscale image
    faces : list
        List of face rectangles [(x, y, w, h), ...]
    elapsed_time : float
        Detection time in seconds
    """
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Histogram equalization for better detection
    gray = cv2.equalizeHist(gray)
    
    # Detect faces
    start_time = time.time()
    faces = cascade.detectMultiScale(
        gray,
        scaleFactor=scale_factor,
        minNeighbors=min_neighbors,
        minSize=min_size,
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    elapsed_time = time.time() - start_time
    
    return gray, faces, elapsed_time


def draw_faces(image: np.ndarray, 
               faces: List, 
               color: Tuple[int, int, int] = (0, 255, 0),
               thickness: int = 2) -> np.ndarray:
    """
    Draw rectangles around detected faces
    
    Parameters:
    -----------
    image : np.ndarray
        Input image
    faces : list
        List of face rectangles
    color : tuple
        Rectangle color in BGR
    thickness : int
        Rectangle line thickness
        
    Returns:
    --------
    output : np.ndarray
        Image with drawn rectangles
    """
    output = image.copy()
    
    for i, (x, y, w, h) in enumerate(faces):
        # Draw rectangle
        cv2.rectangle(output, (x, y), (x+w, y+h), color, thickness)
        
        # Draw face number
        label = f"Face {i+1}"
        cv2.putText(output, label, (x, y-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    
    return output


# 執行人臉檢測
gray, faces_haar, time_haar = detect_faces_haar(img, face_cascade)

# 繪製檢測結果
img_result = draw_faces(img, faces_haar, color=(0, 255, 0))

# 顯示結果
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original Image', fontsize=12)
axes[0].axis('off')

axes[1].imshow(gray, cmap='gray')
axes[1].set_title('Preprocessed (Grayscale + Equalization)', fontsize=12)
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB))
axes[2].set_title(f'Haar Cascade Detection: {len(faces_haar)} faces\nTime: {time_haar*1000:.2f}ms', fontsize=12)
axes[2].axis('off')

plt.tight_layout()
plt.show()

# 輸出詳細信息
print(f"\nDetection Results:")
print(f"Number of faces detected: {len(faces_haar)}")
print(f"Detection time: {time_haar*1000:.2f} ms")
print(f"\nFace coordinates (x, y, w, h):")
for i, (x, y, w, h) in enumerate(faces_haar, 1):
    print(f"Face {i}: ({x}, {y}, {w}, {h})")

### 參數調優實驗

#### 關鍵參數說明

1. **scaleFactor** (1.05 - 1.4)
   - 每次縮放的比例
   - 值越小：檢測越細緻，但速度越慢
   - 推薦：1.1 (平衡) / 1.05 (高精度) / 1.3 (高速度)

2. **minNeighbors** (3 - 6)
   - 每個候選區域需要的鄰居數
   - 值越大：誤檢越少，但可能漏檢
   - 推薦：5 (平衡) / 3 (高召回) / 6 (高精確)

3. **minSize** / **maxSize**
   - 最小/最大人臉尺寸
   - 限制檢測範圍，提升速度
   - 推薦：(30, 30) 最小尺寸

In [ ]:
# 參數調優實驗
param_experiments = [
    {"name": "High Precision", "scale_factor": 1.05, "min_neighbors": 6, "color": (255, 0, 0)},
    {"name": "Balanced", "scale_factor": 1.1, "min_neighbors": 5, "color": (0, 255, 0)},
    {"name": "High Recall", "scale_factor": 1.1, "min_neighbors": 3, "color": (0, 0, 255)},
    {"name": "Fast Detection", "scale_factor": 1.3, "min_neighbors": 4, "color": (255, 255, 0)}
]

results = []

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, params in enumerate(param_experiments):
    # 執行檢測
    gray, faces, elapsed = detect_faces_haar(
        img, face_cascade,
        scale_factor=params["scale_factor"],
        min_neighbors=params["min_neighbors"]
    )
    
    # 繪製結果
    img_result = draw_faces(img, faces, color=params["color"])
    
    # 顯示
    axes[idx].imshow(cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB))
    axes[idx].set_title(
        f"{params['name']}\n"
        f"scaleFactor={params['scale_factor']}, minNeighbors={params['min_neighbors']}\n"
        f"Faces: {len(faces)}, Time: {elapsed*1000:.2f}ms",
        fontsize=11
    )
    axes[idx].axis('off')
    
    results.append({
        "name": params["name"],
        "faces": len(faces),
        "time_ms": elapsed * 1000
    })

plt.tight_layout()
plt.show()

# 輸出比較表
print("\nParameter Tuning Results:")
print("=" * 60)
print(f"{'Configuration':<20} {'Faces':<10} {'Time (ms)':<15}")
print("=" * 60)
for result in results:
    print(f"{result['name']:<20} {result['faces']:<10} {result['time_ms']:<15.2f}")
print("=" * 60)

## 3. LBP Cascade 人臉檢測

### 原理說明

**LBP (Local Binary Patterns)** Cascade 是 Haar Cascade 的輕量級替代方案。

#### 核心概念

1. **LBP 特徵**
   - 局部二值模式
   - 比較中心像素與周圍像素
   - 編碼局部紋理信息

2. **計算流程**
   ```
   1. 取 3x3 鄰域
   2. 比較中心與周圍 8 個像素
   3. 大於中心記為 1，否則記為 0
   4. 得到 8 位二進制數
   ```

#### 優缺點

**優點**:
- 比 Haar Cascade 快 2-3 倍
- 訓練速度快
- 對光照變化更魯棒
- 模型文件更小

**缺點**:
- 準確度略低於 Haar
- 誤檢率稍高
- 對複雜背景敏感

In [ ]:
# 載入 LBP Cascade 分類器
# 注意：OpenCV 的 LBP 模型在不同版本中可能不包含
# 可以從 OpenCV GitHub 下載或使用 Haar 作為替代

try:
    # 嘗試載入 LBP 模型
    lbp_cascade_path = cv2.data.haarcascades + 'lbpcascade_frontalface.xml'
    if not Path(lbp_cascade_path).exists():
        # 嘗試替代路徑
        lbp_cascade_path = cv2.data.haarcascades + 'lbpcascade_frontalface_improved.xml'
    
    face_cascade_lbp = cv2.CascadeClassifier(lbp_cascade_path)
    
    if face_cascade_lbp.empty():
        raise FileNotFoundError("LBP Cascade not found")
    
    LBP_AVAILABLE = True
    print("LBP Cascade classifier loaded successfully")
    print(f"Model path: {lbp_cascade_path}")
    
except Exception as e:
    LBP_AVAILABLE = False
    print(f"LBP Cascade not available: {e}")
    print("Download from: https://github.com/opencv/opencv/tree/master/data/lbpcascades")

In [ ]:
if LBP_AVAILABLE:
    # 執行 LBP 檢測
    gray_lbp, faces_lbp, time_lbp = detect_faces_haar(img, face_cascade_lbp)
    
    # 繪製結果
    img_result_lbp = draw_faces(img, faces_lbp, color=(255, 0, 0))
    
    # Haar vs LBP 比較
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))
    
    axes[0].imshow(cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB))
    axes[0].set_title(f'Haar Cascade\nFaces: {len(faces_haar)}, Time: {time_haar*1000:.2f}ms', fontsize=12)
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(img_result_lbp, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f'LBP Cascade\nFaces: {len(faces_lbp)}, Time: {time_lbp*1000:.2f}ms', fontsize=12)
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # 性能比較
    print("\nHaar vs LBP Comparison:")
    print("=" * 50)
    print(f"{'Metric':<20} {'Haar':<15} {'LBP':<15}")
    print("=" * 50)
    print(f"{'Faces detected':<20} {len(faces_haar):<15} {len(faces_lbp):<15}")
    print(f"{'Time (ms)':<20} {time_haar*1000:<15.2f} {time_lbp*1000:<15.2f}")
    speedup = time_haar / time_lbp
    print(f"{'Speedup':<20} {'1.00x':<15} {f'{speedup:.2f}x':<15}")
    print("=" * 50)
else:
    print("Skipping LBP comparison (LBP cascade not available)")

## 4. HOG + SVM 人臉檢測 (dlib)

### 原理說明

**HOG (Histogram of Oriented Gradients) + SVM** 是 dlib 採用的人臉檢測方法。

#### 核心概念

1. **HOG 特徵**
   - 梯度方向直方圖
   - 描述局部形狀和邊緣信息
   - 對光照變化魯棒

2. **SVM 分類器**
   - Support Vector Machine
   - 線性分類器
   - 最大間隔分類

3. **滑動窗口**
   - 多尺度搜索
   - 金字塔縮放

#### 優缺點

**優點**:
- 準確度高於 Haar/LBP
- 誤檢率低
- 處理側臉效果較好
- 對遮擋有一定魯棒性

**缺點**:
- 速度較慢（比 Haar 慢 5-10 倍）
- 需要額外安裝 dlib
- 資源消耗較大
- 不適合大分辨率圖像

In [ ]:
if DLIB_AVAILABLE:
    # 初始化 dlib HOG 人臉檢測器
    hog_detector = dlib.get_frontal_face_detector()
    print("dlib HOG face detector initialized")
    
    def detect_faces_hog(image: np.ndarray, 
                         detector,
                         upsample_num: int = 1) -> Tuple[np.ndarray, List, float]:
        """
        Detect faces using dlib HOG detector
        
        Parameters:
        -----------
        image : np.ndarray
            Input image (BGR or RGB)
        detector : dlib.fhog_object_detector
            dlib HOG face detector
        upsample_num : int
            Number of times to upsample image (0-2)
            0: fastest, 1: balanced, 2: highest accuracy
            
        Returns:
        --------
        rgb : np.ndarray
            RGB image
        faces : list
            List of face rectangles [(x, y, w, h), ...]
        elapsed_time : float
            Detection time in seconds
        """
        # dlib expects RGB
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Detect faces
        start_time = time.time()
        dets = detector(rgb, upsample_num)
        elapsed_time = time.time() - start_time
        
        # Convert dlib rectangles to (x, y, w, h) format
        faces = []
        for det in dets:
            x = det.left()
            y = det.top()
            w = det.right() - x
            h = det.bottom() - y
            faces.append((x, y, w, h))
        
        return rgb, faces, elapsed_time
    
    # 執行 HOG 檢測
    rgb_hog, faces_hog, time_hog = detect_faces_hog(img, hog_detector, upsample_num=1)
    
    # 繪製結果
    img_result_hog = draw_faces(img, faces_hog, color=(255, 0, 255))
    
    # 三種方法比較
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    axes[0].imshow(cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB))
    axes[0].set_title(f'Haar Cascade\nFaces: {len(faces_haar)}, Time: {time_haar*1000:.2f}ms', fontsize=12)
    axes[0].axis('off')
    
    if LBP_AVAILABLE:
        axes[1].imshow(cv2.cvtColor(img_result_lbp, cv2.COLOR_BGR2RGB))
        axes[1].set_title(f'LBP Cascade\nFaces: {len(faces_lbp)}, Time: {time_lbp*1000:.2f}ms', fontsize=12)
    else:
        axes[1].text(0.5, 0.5, 'LBP Not Available', ha='center', va='center', fontsize=14)
        axes[1].set_title('LBP Cascade', fontsize=12)
    axes[1].axis('off')
    
    axes[2].imshow(cv2.cvtColor(img_result_hog, cv2.COLOR_BGR2RGB))
    axes[2].set_title(f'HOG + SVM (dlib)\nFaces: {len(faces_hog)}, Time: {time_hog*1000:.2f}ms', fontsize=12)
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # 性能比較
    print("\nThree-Method Comparison:")
    print("=" * 70)
    print(f"{'Metric':<20} {'Haar':<15} {'LBP':<15} {'HOG (dlib)':<15}")
    print("=" * 70)
    print(f"{'Faces detected':<20} {len(faces_haar):<15} {len(faces_lbp) if LBP_AVAILABLE else 'N/A':<15} {len(faces_hog):<15}")
    print(f"{'Time (ms)':<20} {time_haar*1000:<15.2f} {time_lbp*1000 if LBP_AVAILABLE else 'N/A':<15} {time_hog*1000:<15.2f}")
    print("=" * 70)
    
else:
    print("dlib not available. Install with: pip install dlib")
    print("Note: dlib requires CMake and C++ compiler")

## 5. 多尺度檢測與參數優化

### 多尺度檢測原理

人臉在圖像中可能有不同大小，多尺度檢測通過圖像金字塔來檢測不同尺度的人臉。

```
原始圖像 → 檢測
縮小 10% → 檢測
縮小 20% → 檢測
...
縮小到最小尺寸 → 檢測
```

### 參數優化策略

In [ ]:
# 創建不同尺度的測試圖像
scales = [0.5, 0.75, 1.0, 1.5]

fig, axes = plt.subplots(2, 4, figsize=(18, 10))

for idx, scale in enumerate(scales):
    # 縮放圖像
    width = int(img.shape[1] * scale)
    height = int(img.shape[0] * scale)
    img_scaled = cv2.resize(img, (width, height))
    
    # 檢測人臉
    gray_scaled, faces_scaled, time_scaled = detect_faces_haar(
        img_scaled, face_cascade,
        scale_factor=1.1,
        min_neighbors=5
    )
    
    # 繪製結果
    img_result_scaled = draw_faces(img_scaled, faces_scaled)
    
    # 顯示原圖
    axes[0, idx].imshow(cv2.cvtColor(img_scaled, cv2.COLOR_BGR2RGB))
    axes[0, idx].set_title(f'Scale {scale}x\nSize: {width}x{height}', fontsize=10)
    axes[0, idx].axis('off')
    
    # 顯示檢測結果
    axes[1, idx].imshow(cv2.cvtColor(img_result_scaled, cv2.COLOR_BGR2RGB))
    axes[1, idx].set_title(f'Detected: {len(faces_scaled)} faces\nTime: {time_scaled*1000:.2f}ms', fontsize=10)
    axes[1, idx].axis('off')

plt.tight_layout()
plt.show()

### 性能優化技巧

#### 1. 圖像預處理

In [ ]:
def preprocess_for_detection(image: np.ndarray, 
                             target_width: int = 640,
                             apply_equalization: bool = True,
                             apply_blur: bool = False) -> Tuple[np.ndarray, float]:
    """
    Preprocess image for faster and more accurate detection
    
    Parameters:
    -----------
    image : np.ndarray
        Input image
    target_width : int
        Resize to this width (keep aspect ratio)
    apply_equalization : bool
        Apply histogram equalization
    apply_blur : bool
        Apply Gaussian blur to reduce noise
        
    Returns:
    --------
    processed : np.ndarray
        Preprocessed image
    scale : float
        Scaling factor applied
    """
    # Resize if image is too large
    height, width = image.shape[:2]
    if width > target_width:
        scale = target_width / width
        new_height = int(height * scale)
        processed = cv2.resize(image, (target_width, new_height))
    else:
        processed = image.copy()
        scale = 1.0
    
    # Convert to grayscale
    if len(processed.shape) == 3:
        processed = cv2.cvtColor(processed, cv2.COLOR_BGR2GRAY)
    
    # Apply histogram equalization
    if apply_equalization:
        processed = cv2.equalizeHist(processed)
    
    # Apply Gaussian blur
    if apply_blur:
        processed = cv2.GaussianBlur(processed, (3, 3), 0)
    
    return processed, scale


# 測試不同預處理方法
preprocessing_configs = [
    {"name": "No Preprocessing", "equalization": False, "blur": False},
    {"name": "Equalization Only", "equalization": True, "blur": False},
    {"name": "Blur Only", "equalization": False, "blur": True},
    {"name": "Both", "equalization": True, "blur": True}
]

fig, axes = plt.subplots(2, 4, figsize=(18, 10))

for idx, config in enumerate(preprocessing_configs):
    # 預處理
    processed, scale = preprocess_for_detection(
        img,
        target_width=640,
        apply_equalization=config["equalization"],
        apply_blur=config["blur"]
    )
    
    # 檢測（注意：這裡直接在灰階圖上檢測）
    start_time = time.time()
    faces_preprocessed = face_cascade.detectMultiScale(
        processed,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )
    time_preprocessed = time.time() - start_time
    
    # 顯示預處理圖像
    axes[0, idx].imshow(processed, cmap='gray')
    axes[0, idx].set_title(f"{config['name']}\nScale: {scale:.2f}x", fontsize=10)
    axes[0, idx].axis('off')
    
    # 繪製檢測結果（在灰階圖上）
    result_gray = cv2.cvtColor(processed, cv2.COLOR_GRAY2BGR)
    result_gray = draw_faces(result_gray, faces_preprocessed)
    
    axes[1, idx].imshow(cv2.cvtColor(result_gray, cv2.COLOR_BGR2RGB))
    axes[1, idx].set_title(f"Faces: {len(faces_preprocessed)}\nTime: {time_preprocessed*1000:.2f}ms", fontsize=10)
    axes[1, idx].axis('off')

plt.tight_layout()
plt.show()

print("\nPreprocessing Impact:")
print("Equalization improves contrast and helps detect faces in poor lighting")
print("Blur reduces noise but may reduce detection accuracy")
print("Resizing to fixed width significantly speeds up detection")

## 6. 性能比較與應用場景

### 綜合性能分析

In [ ]:
# 創建性能比較表
comparison_data = {
    "Method": ["Haar Cascade", "LBP Cascade", "HOG + SVM (dlib)"],
    "Speed": ["Fast (15-30ms)", "Very Fast (8-15ms)", "Slow (50-150ms)"],
    "Accuracy": ["Good", "Fair", "Very Good"],
    "False Positive": ["Medium", "High", "Low"],
    "Side Face": ["Poor", "Poor", "Fair"],
    "Lighting": ["Sensitive", "Robust", "Robust"],
    "Use Case": ["Real-time apps", "Embedded systems", "High accuracy apps"]
}

# 以表格形式顯示
print("\nMethod Comparison:")
print("=" * 120)
print(f"{'Method':<20} {'Speed':<20} {'Accuracy':<15} {'False Positive':<15} {'Side Face':<12} {'Lighting':<12} {'Use Case':<25}")
print("=" * 120)

for i in range(len(comparison_data["Method"])):
    print(f"{comparison_data['Method'][i]:<20} "
          f"{comparison_data['Speed'][i]:<20} "
          f"{comparison_data['Accuracy'][i]:<15} "
          f"{comparison_data['False Positive'][i]:<15} "
          f"{comparison_data['Side Face'][i]:<12} "
          f"{comparison_data['Lighting'][i]:<12} "
          f"{comparison_data['Use Case'][i]:<25}")
print("=" * 120)

### 應用場景推薦

#### 1. 實時視訊監控
- **推薦**: LBP Cascade
- **原因**: 速度最快，資源消耗低

#### 2. 手機 App
- **推薦**: Haar Cascade
- **原因**: 平衡速度與準確度

#### 3. 安全驗證系統
- **推薦**: HOG + SVM
- **原因**: 高準確度，低誤檢率

#### 4. 嵌入式設備
- **推薦**: LBP Cascade
- **原因**: 模型小，計算簡單

#### 5. 大規模照片處理
- **推薦**: Haar Cascade + 並行處理
- **原因**: 可批次處理，速度可接受

## 7. 實戰應用

### 應用 1: 批次圖像人臉檢測

In [ ]:
def batch_face_detection(image_paths: List[str], 
                        cascade: cv2.CascadeClassifier,
                        output_dir: str = None) -> List[dict]:
    """
    Batch face detection for multiple images
    
    Parameters:
    -----------
    image_paths : list
        List of image file paths
    cascade : cv2.CascadeClassifier
        Face detector
    output_dir : str, optional
        Directory to save results
        
    Returns:
    --------
    results : list
        List of detection results
    """
    results = []
    
    for img_path in image_paths:
        if not Path(img_path).exists():
            print(f"Warning: {img_path} not found")
            continue
        
        # Load image
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Failed to load {img_path}")
            continue
        
        # Detect faces
        gray, faces, elapsed = detect_faces_haar(img, cascade)
        
        # Save result
        result = {
            "path": img_path,
            "faces": len(faces),
            "time_ms": elapsed * 1000,
            "coordinates": faces
        }
        results.append(result)
        
        # Save annotated image if output_dir specified
        if output_dir:
            Path(output_dir).mkdir(parents=True, exist_ok=True)
            img_result = draw_faces(img, faces)
            output_path = Path(output_dir) / Path(img_path).name
            cv2.imwrite(str(output_path), img_result)
    
    return results


# 測試批次檢測
test_images = [
    '../assets/images/basic/faces.jpg',
    '../assets/images/basic/face03.jpg',
    '../assets/images/basic/faces01.jpg'
]

# 只處理存在的圖像
existing_images = [p for p in test_images if Path(p).exists()]

if existing_images:
    batch_results = batch_face_detection(
        existing_images,
        face_cascade,
        output_dir='../assets/images/output/face_detection'
    )
    
    # 輸出統計
    print("\nBatch Detection Results:")
    print("=" * 80)
    print(f"{'Image':<40} {'Faces':<10} {'Time (ms)':<15}")
    print("=" * 80)
    
    total_faces = 0
    total_time = 0
    
    for result in batch_results:
        img_name = Path(result['path']).name
        print(f"{img_name:<40} {result['faces']:<10} {result['time_ms']:<15.2f}")
        total_faces += result['faces']
        total_time += result['time_ms']
    
    print("=" * 80)
    print(f"Total: {len(batch_results)} images, {total_faces} faces, {total_time:.2f}ms")
    print(f"Average: {total_time/len(batch_results):.2f}ms per image")
else:
    print("No test images found. Please add images to test batch detection.")

### 應用 2: 人臉提取與裁剪

In [ ]:
def extract_faces(image: np.ndarray, 
                  faces: List,
                  padding: float = 0.2,
                  target_size: Tuple[int, int] = (128, 128)) -> List[np.ndarray]:
    """
    Extract and crop face regions from image
    
    Parameters:
    -----------
    image : np.ndarray
        Input image
    faces : list
        List of face coordinates [(x, y, w, h), ...]
    padding : float
        Padding ratio around face (0.0-0.5)
    target_size : tuple
        Resize face to this size
        
    Returns:
    --------
    face_crops : list
        List of cropped face images
    """
    face_crops = []
    h_img, w_img = image.shape[:2]
    
    for (x, y, w, h) in faces:
        # Add padding
        pad_w = int(w * padding)
        pad_h = int(h * padding)
        
        x1 = max(0, x - pad_w)
        y1 = max(0, y - pad_h)
        x2 = min(w_img, x + w + pad_w)
        y2 = min(h_img, y + h + pad_h)
        
        # Crop face
        face_crop = image[y1:y2, x1:x2]
        
        # Resize
        if target_size:
            face_crop = cv2.resize(face_crop, target_size)
        
        face_crops.append(face_crop)
    
    return face_crops


# 提取人臉
face_crops = extract_faces(img, faces_haar, padding=0.2, target_size=(128, 128))

# 顯示提取的人臉
if face_crops:
    n_faces = len(face_crops)
    n_cols = min(5, n_faces)
    n_rows = (n_faces + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 3*n_rows))
    if n_faces == 1:
        axes = [axes]
    else:
        axes = axes.flatten() if n_rows > 1 else axes
    
    for idx, face_crop in enumerate(face_crops):
        axes[idx].imshow(cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB))
        axes[idx].set_title(f'Face {idx+1}', fontsize=12)
        axes[idx].axis('off')
    
    # Hide unused subplots
    for idx in range(n_faces, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nExtracted {len(face_crops)} face(s)")
    print(f"Face size: {face_crops[0].shape}")
else:
    print("No faces detected to extract")

### 應用 3: 視訊流實時人臉檢測

**注意**: 在 Jupyter Notebook 中實時視訊可能無法正常顯示，建議在獨立 Python 腳本中運行。

In [ ]:
def realtime_face_detection_demo(cascade: cv2.CascadeClassifier,
                                 camera_id: int = 0,
                                 max_frames: int = 100):
    """
    Real-time face detection from webcam (demo version)
    
    Parameters:
    -----------
    cascade : cv2.CascadeClassifier
        Face detector
    camera_id : int
        Camera device ID
    max_frames : int
        Maximum frames to process (for demo)
    
    Note:
    -----
    This is a demo function. For actual real-time detection,
    run as a standalone Python script.
    """
    print("Real-time Face Detection Demo")
    print("=" * 50)
    print("This function demonstrates real-time detection code.")
    print("To run actual webcam detection:")
    print("1. Save this code to a .py file")
    print("2. Run: python face_detection_realtime.py")
    print("3. Press 'q' to quit")
    print("=" * 50)
    
    # Demo code (commented out for notebook)
    demo_code = '''
import cv2
import time

# Load cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open webcam
cap = cv2.VideoCapture(0)

# FPS calculation
fps_list = []
frame_count = 0

while True:
    start_time = time.time()
    
    # Read frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Preprocess
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )
    
    # Draw results
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, "Face", (x, y-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    
    # Calculate FPS
    elapsed = time.time() - start_time
    fps = 1 / elapsed if elapsed > 0 else 0
    fps_list.append(fps)
    avg_fps = sum(fps_list[-30:]) / len(fps_list[-30:])
    
    # Display info
    cv2.putText(frame, f"Faces: {len(faces)}", (10, 30),
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"FPS: {avg_fps:.1f}", (10, 60),
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # Show frame
    cv2.imshow('Face Detection', frame)
    
    # Check for quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    frame_count += 1

# Cleanup
cap.release()
cv2.destroyAllWindows()

print(f"Processed {frame_count} frames")
print(f"Average FPS: {sum(fps_list)/len(fps_list):.2f}")
    '''
    
    print("\nExample Code:")
    print(demo_code)

# 顯示 demo 說明
realtime_face_detection_demo(face_cascade)

### 應用 4: 多人臉追蹤與編號

In [ ]:
def track_faces_simple(faces_prev: List, 
                       faces_curr: List,
                       max_distance: int = 50) -> List[Tuple[int, Tuple]]:
    """
    Simple face tracking by distance matching
    
    Parameters:
    -----------
    faces_prev : list
        Previous frame faces [(id, (x, y, w, h)), ...]
    faces_curr : list
        Current frame faces [(x, y, w, h), ...]
    max_distance : int
        Maximum distance for matching
        
    Returns:
    --------
    tracked : list
        Tracked faces [(id, (x, y, w, h)), ...]
    """
    if not faces_prev:
        # First frame: assign new IDs
        return [(i, face) for i, face in enumerate(faces_curr)]
    
    tracked = []
    used_ids = set()
    next_id = max([fid for fid, _ in faces_prev]) + 1
    
    for curr_face in faces_curr:
        x_curr, y_curr, w_curr, h_curr = curr_face
        cx_curr = x_curr + w_curr // 2
        cy_curr = y_curr + h_curr // 2
        
        # Find closest previous face
        min_dist = float('inf')
        matched_id = None
        
        for prev_id, prev_face in faces_prev:
            if prev_id in used_ids:
                continue
            
            x_prev, y_prev, w_prev, h_prev = prev_face
            cx_prev = x_prev + w_prev // 2
            cy_prev = y_prev + h_prev // 2
            
            # Euclidean distance
            dist = np.sqrt((cx_curr - cx_prev)**2 + (cy_curr - cy_prev)**2)
            
            if dist < min_dist and dist < max_distance:
                min_dist = dist
                matched_id = prev_id
        
        # Assign ID
        if matched_id is not None:
            tracked.append((matched_id, curr_face))
            used_ids.add(matched_id)
        else:
            # New face
            tracked.append((next_id, curr_face))
            next_id += 1
    
    return tracked


print("Face tracking function defined.")
print("This can be used for video processing to maintain face IDs across frames.")
print("\nExample usage:")
print("""
tracked_faces = []
for frame in video:
    faces = detect_faces_haar(frame, face_cascade)
    tracked_faces = track_faces_simple(tracked_faces, faces)
    # Draw with IDs...
""")

## 8. 進階主題與深度學習預覽

### 8.1 級聯分類器訓練基礎

#### 訓練自己的 Haar Cascade

雖然 OpenCV 提供了預訓練模型，但有時需要訓練自定義檢測器（如特定姿勢、特定物體）。

**訓練步驟**:

1. **準備數據集**
   - 正樣本（包含目標物體）: 1000-5000 張
   - 負樣本（不含目標）: 3000-10000 張

2. **創建樣本描述文件**
   ```bash
   opencv_createsamples -info positives.txt -vec samples.vec -w 24 -h 24
   ```

3. **訓練級聯分類器**
   ```bash
   opencv_traincascade -data cascade/ -vec samples.vec -bg negatives.txt \
       -numPos 800 -numNeg 400 -numStages 20 -w 24 -h 24
   ```

4. **測試與優化**
   - 調整 numStages
   - 調整 minHitRate 和 maxFalseAlarmRate

### 8.2 誤檢與漏檢處理

#### 減少誤檢 (False Positives)

1. **提高 minNeighbors** - 更嚴格的檢測標準
2. **限制檢測區域** - 只在可能出現人臉的區域檢測
3. **多檢測器投票** - 使用多個檢測器，只接受多數同意的結果
4. **後處理過濾** - 根據人臉特徵（長寬比、膚色等）過濾

In [ ]:
def filter_false_positives(image: np.ndarray, 
                          faces: List,
                          aspect_ratio_range: Tuple[float, float] = (0.75, 1.3),
                          min_area: int = 900) -> List:
    """
    Filter false positive detections
    
    Parameters:
    -----------
    image : np.ndarray
        Input image
    faces : list
        Detected faces [(x, y, w, h), ...]
    aspect_ratio_range : tuple
        Valid aspect ratio range (w/h)
    min_area : int
        Minimum face area (w*h)
        
    Returns:
    --------
    filtered : list
        Filtered faces
    """
    filtered = []
    
    for (x, y, w, h) in faces:
        # Check aspect ratio
        aspect_ratio = w / h if h > 0 else 0
        if not (aspect_ratio_range[0] <= aspect_ratio <= aspect_ratio_range[1]):
            continue
        
        # Check area
        area = w * h
        if area < min_area:
            continue
        
        # Additional checks can be added here:
        # - Skin color detection
        # - Edge density
        # - Symmetry
        
        filtered.append((x, y, w, h))
    
    return filtered


# 測試過濾
faces_filtered = filter_false_positives(
    img, faces_haar,
    aspect_ratio_range=(0.8, 1.2),
    min_area=900
)

print(f"\nFalse Positive Filtering:")
print(f"Original detections: {len(faces_haar)}")
print(f"After filtering: {len(faces_filtered)}")
print(f"Removed: {len(faces_haar) - len(faces_filtered)}")

# 視覺化比較
if len(faces_haar) != len(faces_filtered):
    img_before = draw_faces(img, faces_haar, color=(0, 0, 255))
    img_after = draw_faces(img, faces_filtered, color=(0, 255, 0))
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))
    
    axes[0].imshow(cv2.cvtColor(img_before, cv2.COLOR_BGR2RGB))
    axes[0].set_title(f'Before Filtering: {len(faces_haar)} faces', fontsize=12)
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(img_after, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f'After Filtering: {len(faces_filtered)} faces', fontsize=12)
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()

### 8.3 深度學習方法預覽

#### 現代深度學習人臉檢測

傳統方法的局限性促使深度學習方法的發展：

1. **OpenCV DNN 模組**
   - 使用預訓練的 Caffe/TensorFlow 模型
   - SSD (Single Shot Detector)
   - ResNet 基礎網絡

2. **MTCNN (Multi-task Cascaded CNN)**
   - 三階段級聯網絡
   - 同時檢測人臉和關鍵點
   - 速度與準確度平衡

3. **RetinaFace**
   - SOTA 性能
   - 處理極小人臉
   - 多任務學習

4. **YOLOv5-Face / YOLOv8-Face**
   - 實時性能
   - 易於部署
   - 準確度高

In [ ]:
# OpenCV DNN 人臉檢測示例（如果有模型）
dnn_model_path = '../assets/models/res10_300x300_ssd_iter_140000_fp16.caffemodel'
dnn_config_path = '../assets/models/deploy.prototxt'

if Path(dnn_model_path).exists() and Path(dnn_config_path).exists():
    print("OpenCV DNN face detection model found")
    
    # Load DNN model
    net = cv2.dnn.readNetFromCaffe(dnn_config_path, dnn_model_path)
    
    def detect_faces_dnn(image: np.ndarray, 
                         net,
                         confidence_threshold: float = 0.5) -> Tuple[List, float]:
        """
        Detect faces using OpenCV DNN
        
        Parameters:
        -----------
        image : np.ndarray
            Input image
        net : cv2.dnn.Net
            DNN model
        confidence_threshold : float
            Minimum confidence (0.0-1.0)
            
        Returns:
        --------
        faces : list
            Detected faces [(x, y, w, h), ...]
        elapsed_time : float
            Detection time
        """
        h, w = image.shape[:2]
        
        # Preprocess
        blob = cv2.dnn.blobFromImage(
            cv2.resize(image, (300, 300)),
            1.0,
            (300, 300),
            (104.0, 177.0, 123.0)
        )
        
        # Detect
        start_time = time.time()
        net.setInput(blob)
        detections = net.forward()
        elapsed_time = time.time() - start_time
        
        faces = []
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            
            if confidence > confidence_threshold:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x1, y1, x2, y2) = box.astype("int")
                
                # Convert to (x, y, w, h)
                x = max(0, x1)
                y = max(0, y1)
                w = min(w, x2) - x
                h = min(h, y2) - y
                
                faces.append((x, y, w, h))
        
        return faces, elapsed_time
    
    # 執行 DNN 檢測
    faces_dnn, time_dnn = detect_faces_dnn(img, net, confidence_threshold=0.5)
    img_result_dnn = draw_faces(img, faces_dnn, color=(0, 255, 255))
    
    # 四種方法比較
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    methods = [
        ("Haar Cascade", img_result, len(faces_haar), time_haar),
        ("LBP Cascade", img_result_lbp if LBP_AVAILABLE else img, len(faces_lbp) if LBP_AVAILABLE else 0, time_lbp if LBP_AVAILABLE else 0),
        ("HOG + SVM (dlib)", img_result_hog if DLIB_AVAILABLE else img, len(faces_hog) if DLIB_AVAILABLE else 0, time_hog if DLIB_AVAILABLE else 0),
        ("DNN (Deep Learning)", img_result_dnn, len(faces_dnn), time_dnn)
    ]
    
    for idx, (name, result_img, n_faces, elapsed) in enumerate(methods):
        axes[idx].imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
        axes[idx].set_title(f'{name}\nFaces: {n_faces}, Time: {elapsed*1000:.2f}ms', fontsize=12)
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("\nDeep Learning Comparison:")
    print("DNN method typically offers:")
    print("  + Higher accuracy (especially for challenging poses)")
    print("  + Better handling of occlusions")
    print("  + More robust to lighting variations")
    print("  - Slower than traditional methods (but still real-time capable)")
    print("  - Requires model files (larger storage)")
    
else:
    print("\nDNN models not found.")
    print("For deep learning face detection, you need:")
    print("1. res10_300x300_ssd_iter_140000_fp16.caffemodel")
    print("2. deploy.prototxt")
    print("\nThese are typically included in the assets/models/ directory.")

### 深度學習 vs 傳統方法對比

| 特性 | 傳統方法 (Haar/LBP/HOG) | 深度學習 (CNN/DNN) |
|-----|------------------------|-------------------|
| 準確度 | 中等 (85-90%) | 高 (95-99%) |
| 速度 | 快 (10-50ms) | 中等 (20-100ms) |
| 側臉檢測 | 差 | 優秀 |
| 遮擋處理 | 差 | 良好 |
| 光照魯棒性 | 中等 | 優秀 |
| 訓練難度 | 高 | 中等（有預訓練模型） |
| 資源需求 | 低 | 中-高 |
| 部署難度 | 簡單 | 中等 |
| 模型大小 | 小 (KB-MB) | 大 (MB-GB) |

### 選擇建議

- **嵌入式/移動設備**: 傳統方法 (Haar/LBP)
- **高準確度要求**: 深度學習
- **實時性要求**: 取決於硬件，現代 GPU 可支持 DNN 實時
- **開發成本**: 傳統方法更易上手
- **長期維護**: 深度學習模型更易更新和改進

## 9. 練習題

### 練習 1: 參數調優實驗

使用不同參數組合，找到最佳的檢測設置：
- 測試 scaleFactor: 1.05, 1.1, 1.2, 1.3
- 測試 minNeighbors: 3, 4, 5, 6
- 記錄檢測數量和時間
- 找到速度與準確度的最佳平衡點

In [ ]:
# TODO: 實作參數調優實驗
# 提示：
# 1. 創建嵌套循環遍歷所有參數組合
# 2. 記錄每個組合的結果
# 3. 創建熱力圖顯示性能
# 4. 分析最佳參數

pass

### 練習 2: 多檢測器融合

結合 Haar 和 LBP 檢測器，提高檢測可靠性：
- 使用兩個檢測器分別檢測
- 只保留兩者都檢測到的人臉（交集）
- 比較融合前後的誤檢率

In [ ]:
# TODO: 實作多檢測器融合
# 提示：
# 1. 定義 IoU (Intersection over Union) 函數
# 2. 匹配兩個檢測器的結果
# 3. 只保留高 IoU 的檢測

pass

### 練習 3: 視訊人臉檢測與統計

處理一段視訊，統計人臉出現情況：
- 檢測每一幀的人臉
- 追蹤人臉 ID
- 統計出現次數
- 繪製人臉數量隨時間變化的曲線

In [ ]:
# TODO: 實作視訊人臉統計
# 提示：
# 1. 讀取視訊文件
# 2. 逐幀檢測並追蹤
# 3. 記錄統計數據
# 4. 繪製統計圖表

pass

### 練習 4: 人臉質量評分

為檢測到的人臉評分（適合做人臉識別的程度）：
- 檢查清晰度（邊緣強度）
- 檢查正臉程度（眼睛檢測）
- 檢查光照均勻度（直方圖）
- 綜合評分 0-100

In [ ]:
# TODO: 實作人臉質量評分
# 提示：
# 1. 提取人臉區域
# 2. 計算各項質量指標
# 3. 加權平均得到總分
# 4. 只保留高分人臉

pass

## 10. 總結

### 關鍵要點

1. **三種經典方法**
   - Haar Cascade: 平衡速度與準確度
   - LBP Cascade: 最快速度
   - HOG + SVM: 最高準確度（傳統方法中）

2. **參數調優重要性**
   - scaleFactor: 控制檢測細緻度
   - minNeighbors: 控制誤檢率
   - 需根據應用場景調整

3. **預處理技巧**
   - 灰階化 + 直方圖均衡化
   - 縮放到合適尺寸
   - 可選的模糊降噪

4. **實戰考慮**
   - 速度 vs 準確度權衡
   - 誤檢與漏檢處理
   - 實時性能優化
   - 多人臉追蹤

5. **未來趨勢**
   - 深度學習方法主流化
   - 端側部署優化
   - 多任務聯合學習
   - 隱私保護檢測

### 性能基準

在標準測試圖像上（640x480 像素）：
- Haar Cascade: ~15-30ms
- LBP Cascade: ~8-15ms
- HOG + SVM: ~50-150ms
- DNN (CPU): ~30-80ms
- DNN (GPU): ~5-15ms

### 延伸學習

1. **人臉對齊** (Face Alignment)
   - 68 點關鍵點檢測
   - Affine 變換標準化

2. **人臉識別** (Face Recognition)
   - 特徵提取（FaceNet, ArcFace）
   - 相似度計算
   - 1:1 驗證與 1:N 識別

3. **人臉屬性分析**
   - 年齡估計
   - 性別識別
   - 表情識別
   - 種族預測

4. **3D 人臉重建**
   - 深度估計
   - 姿態估計
   - 3D Mesh 生成

### 參考資源

- OpenCV Face Detection Tutorial: https://docs.opencv.org/4.x/db/d28/tutorial_cascade_classifier.html
- Viola-Jones Paper: https://www.cs.cmu.edu/~efros/courses/LBMV07/Papers/viola-cvpr-01.pdf
- dlib Documentation: http://dlib.net/face_detector.py.html
- OpenCV DNN Module: https://docs.opencv.org/4.x/d2/d58/tutorial_table_of_content_dnn.html
- Face Detection Benchmark: http://vis-www.cs.umass.edu/fddb/

---

## 下一步

完成本模組後，建議繼續學習：
- **5.1.2 人臉對齊** - 關鍵點檢測與標準化
- **5.1.3 人臉識別** - 特徵提取與相似度計算
- **5.2.1 物體檢測基礎** - YOLO 與 SSD

**實戰項目建議**:
1. 智能相冊：自動標記照片中的人臉
2. 人流統計：視訊監控中的人數統計
3. 注意力檢測：學生課堂專注度監測
4. 智能門禁：人臉檢測 + 識別系統

---

**模組完成標記**: ✅ WBS 5.1.1 Face Detection Complete